<a href="https://colab.research.google.com/github/freejae98/freejae98.github.io/blob/master/2022_10_18_f_2)_%EC%B2%A0%EA%B0%95_ML_colab_22_1018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:

# warning 무시 코드 
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')


############################################################################################################

# import win32com.client as win32
import pandas as pd 
import numpy as np
# import matplotlib.pyplot as plt    # 그래프를 그리는 패키지

# from matplotlib import font_manager, rc
# font_name = font_manager.FontProperties(fname="C:/Windows/Fonts/Malgunsl.ttf").get_name()
# rc('font', family=font_name)

############################################################################################################



import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Flatten, Dropout
from keras.layers import SimpleRNN, LSTM, GRU

from sklearn.preprocessing import MinMaxScaler

from pickle import load      # scaler load 목적

import openpyxl


# 예측대상 변수 연결 
predict_what = '2)열연-중국천진'

In [13]:


# =============================================================================
# # 1) model, scaler 불러오기 (4개 모델/scaler)   * scaler는 동일함
# =============================================================================
model_1 = keras.models.load_model("/content/drive/MyDrive/ML 모델/steel_1d.h5")
model_2 = keras.models.load_model("/content/drive/MyDrive/ML 모델/steel_2d.h5")
model_3 = keras.models.load_model("/content/drive/MyDrive/ML 모델/steel_3d.h5")
model_4 = keras.models.load_model("/content/drive/MyDrive/ML 모델/steel_4d.h5")

# model_1.summary()

scaler = load(open('/content/drive/MyDrive/ML 모델/scaler_1d.pkl', 'rb'))

In [14]:

# =============================================================================
# # 2) data_set 읽어오기 - ndf1 로 받아옴
# =============================================================================
fname1 = r"/content/drive/MyDrive/(Total) Data_Lake.xlsx"      # data lake를 가져옴


df1 = pd.read_excel(fname1)

ndf1 = df1.astype({'Date':'datetime64'})
ndf1.set_index('Date', inplace=True)

ndf1.info()
ndf1.head()




# =============================================================================
# 3-2) for 돌리기 - 테스트
# =============================================================================


sequence = 15
predict_day = 1   #5*1 # 1주 후


# 데이터 구성
# X_colname = ['MA5']
raw_all = ndf1['MA5']


raw_all = ndf1.loc[:, 'MA5']
raw_all = raw_all[-sequence:]

# raw_all.info()
# raw_all.shape


raw_all = pd.DataFrame(raw_all)

raw_scaled = scaler.transform(raw_all)
raw_scaled.shape



# 4주차 각각으로 예측하는 부분 - Y_result_i : 리스트로 생성
X_real, Y_result, Y_result_i = [], [], []

for index in range(0, len(raw_scaled) - sequence + 1):
    # print(index)
    X_real.append(raw_scaled[index: index + sequence])


print(X_real)

X_real = np.array(X_real)

X_real.shape
# (1, 15, 1)


for i_model in [model_1, model_2, model_3, model_4]:
    
    Y_pred = i_model.predict(X_real)
    Y_pred_i = scaler.inverse_transform(Y_pred)   
    
    
    Y_result.append(Y_pred[0,0])
    Y_result_i.append(Y_pred_i[0,0])





# 일단 데이터 정확도는 고려하지 않고, 데이터 업데이트 하는 부분 해보자!
# Y_result_i 를 먼저 date와 결합한 df형태로 만든 다음에, MA5와 결합하는 것으로 진행하면 될 듯 함
from datetime import datetime, timedelta




# =============================================================================
# # 4) 예측된 값을 파일에 업데이트 해주기
# 
#  - 최종 붙일 데이터셋은 result_t5, 이걸 ndf1에 붙이면 됨
# =============================================================================


# 주 단위 날짜 4개 data만 만들어서 업데이트 해줌 (주단위, 마지막 date 기준)


temp = raw_all.index[-1]


start_date = temp + timedelta(days=7)
end_date = start_date + timedelta(days=21)
# Timestamp('2022-06-03 00:00:00')


# start_date = pd.to_datetime('2015-12-20') ## 시작 날짜  (MA5 만들기 위해 좀더 앞으로 땡김)
# end_date = pd.to_datetime('2022-06-02') ## 마지막 날짜
  
## 시작 날짜와 마지막 날짜 사이에 날짜를 생성하고 이를 인덱스로 설정
t_date = pd.date_range(start_date,end_date,freq='7d')

t_date = pd.DataFrame(t_date, columns=['Date'])
t_date = t_date.astype({'Date':'datetime64[ns]'})


t_date_reset = t_date.reset_index(drop=True)


# Y_result_df = pd.DataFrame(Y_result_i, columns=['Prediction'])
Y_result_sr = pd.Series(Y_result_i, name = predict_what)       #예측대상 변수 연결 

# len(Y_result_i)

# result_t = pd.concat([t_date,Y_result_df], axis=1)

result_t = pd.concat([t_date_reset,Y_result_sr], axis=1)

# result_t2 = result_t.dropna(subset=['Prediction'], how='any', axis=0)
# len(result_t2)

# result_t3 = result_t2.drop('week', axis=1)


result_t.set_index('Date', inplace=True)




# =============================================================================
# # 마지막 하이라이트, 데이터셋의 최종 결합
# =============================================================================
final_ds = pd.concat([ndf1, result_t])

# final_ds.to_excel('D:/★프로세스 혁신★/22.01월/1. AI 업체, 데이터분석 조사/4. 데이터 연결/data_completed_2.xlsx', index=True)


final_ds.to_excel('/content/drive/MyDrive/(Final) Data_Lake.xlsx', sheet_name='원본', index=True)
final_ds.info()


# 새로운 테스트
# 그냥 파일은 만들고.. 여기서 '원본', 다른 파일에서 '원본세팅' 읽어와서 하나의 새 파일로 조합하는게 훨 낫겠네~~
path_ref = '/content/drive/MyDrive/(step2) reference_T.xlsx'



ref_df = pd.read_excel(path_ref) 

nref_df = ref_df.iloc[:,1:]


# ndf1 = df1.astype({'Date':'datetime64[ns]'})
# ndf1.set_index('Date', inplace=True)

# # ndf1.describe(include='all')
# # ndf1 = ndf1.drop('6)Contract', axis=1)


# ndf1 = ndf1.astype('float')

nref_df.info()


output_file = '/content/drive/MyDrive/(최종) 태블로용 연결파일.xlsx'



# output_file = path_1  #'D:/★프로세스 혁신★/22.09월/철강 MI 실행 추진/철강MI_데이터 수집/data_collected.xlsx'

f_writer=pd.ExcelWriter(output_file, engine='openpyxl')


final_ds.to_excel(f_writer, index=True, sheet_name="원본")
nref_df.to_excel(f_writer, index=True, sheet_name="원본 세팅")

f_writer.save()




<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1774 entries, 2015-12-25 to 2022-10-17
Columns: 116 entries, 1)환율-RMB to MA5
dtypes: float64(112), int64(4)
memory usage: 1.6 MB
[array([[0.28461043],
       [0.28074694],
       [0.27656149],
       [0.27237605],
       [0.27108822],
       [0.2704443 ],
       [0.2704443 ],
       [0.2704443 ],
       [0.2704443 ],
       [0.27108822],
       [0.27012234],
       [0.26786864],
       [0.26529298],
       [0.26271732],
       [0.25820992]])]
1/1 [==============================] - 0s 382ms/step
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1778 entries, 2015-12-25 to 2022-11-14
Columns: 116 entries, 1)환율-RMB to MA5
dtypes: float64(116)
memory usage: 1.6 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   국가           79 non-null     object 
 1   특성           99 non-null     object 
 2